# Exercise 9 - 2025: Training the Lennard-Jones, training with the Lennard-Jones. 

In this exercise we want to observe the top panel of Figure 2 of [this paper](https://aip.scitation.org/doi/pdf/10.1063/1.481671) and see if we recover the same trend, that is a disordering of a 38 atom LJ cluster.
We will do that by performing simulations at various temperatures with LAMMPS, extracting the order parameters Q4 and Q6, and extracting averages.
We then will use some of the MD directories to train a NN potential. This is just the first part of the exercise, in a few weeks we will go deeper and all assignments will concern the two notebooks, together.

In [ ]:
import numpy as np
import scipy
from scipy.special import sph_harm
from glob import glob
from ase.io import read,write
from ase import neighborlist
import matplotlib.pyplot as plt

import numpy as np
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv


import ase
import ase.io
import ase.lattice.cubic
import ase.md
from ase.md.nvtberendsen import NVTBerendsen
from ase.units import fs, kB
from ase.calculators.lammpsrun import LAMMPS

def view_structure(system):
    t = nv.ASEStructure(system) 
    w = nv.NGLWidget(t, gui=True)
    w.add_spacefill()
    return w


def view_trajectory(trajectory):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    w2.add_spacefill(radius=0.1)
    return w2


global epsilon_lammps,sigma_lammps


epsilon_lammps = 0.01042
sigma_lammps = 3.405

eps_kelvin = (epsilon_lammps/kB) # value for argon

print (f"The value for argon of the epsilon, in K is {eps_kelvin}")

### 3.2. Bond order parameters

We now need to create a function for the calculation of bond order parameters (or Steinhardt order parameters), which we have already met in Exercise 3. As a short recap, they are defined as:

$$
Q_l^2 = \frac{4 \pi}{2l+1} \sum_{m=-l}^l |q_{lm}|^2
\qquad
 \text{with}:
 \quad
 q_{lm} = \frac{1}{N_{bonds}} \sum_{N_n} Y_{lm} (\theta_{ij}, \phi_{ij}).
$$

These parameters hold the information of the local structure and are sensitive to different symmetries. Just as in Exercise 3, the center of mass of the full 38-atom cluster is calculated, and thereafter the single nearest neighbour atom to the center of mass is selected. This nearest neighbour plus its own 12 nearest neighbours themselves define the "core" of the total 38-atom cluster. This process is repeated over all vectors pointing from the center of mass of the 38-atom cluster to all $N_b$ bonds over the formed over the  We first define a general `Python` function for $Q_l$ calculation, and then compute $Q_4$ for all the computed trajectories by simply calling the function. Execute the code block which defines a function `QL`, which is of course identical to that in Exercise 3:

In [ ]:
def QL(l, trajectory):
    """
    Calculate the QL order parameter for a given trajectory.

    Args:
        l (int): the order of the QL parameter; corresponds to the l in the spherical harmonics
        trajectory (ase.Atoms): the trajectory to be analyzed
    
    Returns:
        Ql (np.array): the Ql order parameter for each frame of the trajectory
    """
    
    rcut = sigma_lammps*1.391/2 #in Angstrom
    
    #update the cutoff for each frame
    for frame in trajectory: 
        array_rcut         = np.ones(len(frame))*rcut
        new_neighbour_list = neighborlist.NeighborList(array_rcut,skin=0, self_interaction=False, bothways=False)
        new_neighbour_list.update(frame)
    
    
    #compute Ql for each frame
    Ql = np.empty(len(trajectory))
    i  = 0
    
    for frame in trajectory:
        nbonds = 0
        qlm    = np.zeros(2*l+1) 
        
        for atom in frame:
            nlist = new_neighbour_list.get_neighbors(atom.index)[0]
                
            for theneig in nlist:   #cycle over the neighbours
                #get angles and distances
                nbonds   = nbonds+1
                rij      = frame[theneig].position - atom.position
                dist     = np.linalg.norm(rij)
                phi_ij   = np.arccos(rij[2]/dist)
                theta_ij = np.arctan2(rij[1],rij[0])
                if theta_ij < 0:
                    theta_ij += 2*np.pi
                
                #move in spherical coordinates space
                    # In a like-oriented coordinate system at j,
                    #the spherical coordinates of atom i are:
                if theta_ij  <= np.pi:
                    theta_ji  = theta_ij + np.pi
                elif theta_ij > np.pi:
                    theta_ji  = theta_ij - np.pi
                if np.absolute(theta_ji-2*np.pi)<0.0001:
                    theta_ji=0.0
                phi_ji = np.pi-phi_ij
                
                #compute spherical harmonics and perform qml summation 
                qlm = qlm + np.array([ sph_harm(m,l,theta_ij,phi_ij) for m in range(-l,l+1) ])
                
        qlm   =  np.real(qlm*np.conj(qlm)/(nbonds*nbonds))
        
        #prefactor and second summation
        Ql[i] =  np.sqrt(np.pi *4 /(2*l+1)*np.sum(qlm))
        i    += 1
    return Ql

# Initial structure

In [ ]:
structure_t0 = read ("T0.xyz")

view_structure (structure_t0)

## A LAMMPS ASE calculator would look like that, but is MUCH slower than the command line!

In [ ]:


atoms = read ("T0.xyz")

atoms.set_cell([50, 50, 50])  # Define box size
atoms.center()  # Center atoms in the box
atoms.set_pbc([True,True,True])  # Enable periodic boundaries
parameters = {"pair_style": "lj/cut 8.5",
              "pair_coeff": ['1 1 0.01042 3.405  8.5'],
              "units":"metal",
              "atom_style":"atomic",}



"""



# Set up LAMMPS calculator
calc = LAMMPS(command ="lmp_serial", label="argon_md", **parameters)
atoms.calc = calc
atoms.write("argon.data", format="lammps-data")



print (atoms.get_potential_energy())

print (atoms.get_positions())
# Define molecular dynamics (MD) parameters
timestep = 1.0  # femtoseconds
temperature = 300  # Kelvin

dyn = NVTBerendsen(atoms, timestep=1.0 * fs, temperature_K=300, taut=0.1)


from ase.io.trajectory import Trajectory

traj = Trajectory("argon.traj", "w", atoms)

#for step in range(10000):
#    dyn.run(1)
#    if step % 1000 == 0:
#        print (step,atoms.get_potential_energy())
#        traj.write(atoms)


"""


## Example calculation at 22 K

In [ ]:
import ase
import ase.io
import ase.lattice.cubic
import ase.md
from ase.md.nvtberendsen import NVTBerendsen
from ase.units import fs, kB
from ase.calculators.lammpsrun import LAMMPS

atoms = read ("T0.xyz")

atoms.set_cell([50, 50, 50])  # Define box size
atoms.center()  # Center atoms in the box
atoms.set_pbc([True,True,True])  # Enable periodic boundaries
atoms.write("T0.data", format="lammps-data")

temp = 22

placeholder = "{PLACEHOLDER}"  # Define the placeholder to be replaced
replacement = str(temp)  # The new content for the placeholder
file_path = "input.T0"  # Path to your existing file
file_path2 = "input.T"
!rm {file_path2}
# Read the file
with open(file_path, "r") as file:
    content = file.read()

# Replace the placeholder
updated_content = content.replace(placeholder, replacement)

# Write the modified content back to the file
with open(file_path2, "w") as file2:
    file2.write(updated_content)





!lmp_serial -i input.T > out.{replacement}



In [ ]:
filename = f"structure_{temp}.xyz"
traj = read(filename,index=":")
print (f"Trajectory for T={temp}")
view_trajectory (traj)

Now execute the code block defining `Q4` and `Q6`:

In [ ]:

trajectory = read(filename, index=":")

import numpy as np
import matplotlib.pyplot as plt
q4_values = QL(4,trajectory)
# Plot Q4 along the trajectory
plt.plot(q4_values, marker="o")
plt.xlabel("Frame Index")
plt.ylabel("Q4 Value")
title = f"Q4 Structural Parameter, T = {temp}"
plt.title(title)
plt.show()

q6_values = QL(6,trajectory)
# Plot Q4 along the trajectory
plt.plot(q6_values, marker="o")
plt.xlabel("Frame Index")
plt.ylabel("Q6 Value")
title = f"Q6 Structural Parameter, T = {temp}"
plt.title(title)
plt.show()



## Cycle over many temperatures (one million steps)

In [ ]:
temperature_values = np.array([xxxxxxxxxxxxxxxxxxxxxx])  #insert the temperatures appropriate (in K) looking at the paper. Discussion on units. 
 

q4_values = [[] for _ in temperature_values]
q6_values = [[] for _ in temperature_values]

! rm structure_*.xyz 
! cat input.T0


for i,temp in enumerate(temperature_values):
    placeholder = "{PLACEHOLDER}"  # Define the placeholder to be replaced
    temp_str = str(temp)  # The new content for the placeholder
    file_path = "input.T0"  # Path to your existing file
    file_path2 = "input.T"
    !rm {file_path2}
    # Read the file
    print (f"Running {temp}")
    with open(file_path, "r") as file:
        content = file.read()

    # Replace the placeholder
    updated_content = content.replace(placeholder, temp_str)

    # Write the modified content back to the file
    with open(file_path2, "w") as file2:
        file2.write(updated_content)


    

    !lmp_serial -i input.T > out.{temp_str}

    my_str = f"structure_{temp_str}.xyz"

    trajectory = read(my_str, index=":")
    
    
    import numpy as np
    import matplotlib.pyplot as plt
    q4_values[i] = QL(4,trajectory)
    q6_values[i] = QL(6,trajectory)
""" # Plot Q4 along the trajectory
    plt.plot(q4_values[i], marker="o")
    plt.xlabel("Frame Index")
    plt.ylabel("Q4 Value")
    title = f"Q4 Structural Parameter {temp_str}"
    plt.title(title)
    plt.show()


    # Plot Q4 along the trajectory
    plt.plot(q6_values[i], marker="o")
    plt.xlabel("Frame Index")
    title = f"Q6 Structural Parameter {temp_str}"
    plt.title(title)
    plt.ylabel("Q6 Value")
    plt.show()"""
    
    
    

data = np.column_stack((temperature_values, q4_values, q6_values))    


print ("Set of simulations finished! Temperatures: ",temperature_values)


# Compute the average for each temperature


In [ ]:
q4_averages = [np.mean(q4) for q4 in q4_values]

# Plot the results
plt.figure(figsize=(8, 5))

t_values = temperature_values * 1./eps_kelvin
plt.plot(t_values, q4_averages, marker='o', linestyle='-')

# Labels and title
plt.xlabel("Temperature/eps)")
plt.ylabel("Average q4")
plt.title("Average q4 vs Temperature")
plt.grid(True)

# Show the plot
plt.show()


plt.plot(temperature_values, q4_averages, marker='o', linestyle='-')

# Labels and title
plt.xlabel("Temperature)")
plt.ylabel("Average q4")
plt.title("Average q4 vs Temperature")
plt.grid(True)

# Show the plot
plt.show()

# Compute the average for each temperature
q6_averages = [np.mean(q6) for q6 in q6_values]

# Plot the results
plt.figure(figsize=(8, 5))

t_relative = temperature_values * 1./eps_kelvin
plt.plot(t_relative, q6_averages, marker='o', linestyle='-')

# Labels and title
plt.xlabel("Temperature/eps)")
plt.ylabel("Average q6")
plt.title("Average q6 vs Temperature")
plt.grid(True)

# Show the plot
plt.show()


plt.plot(temperature_values, q6_averages, marker='o', linestyle='-')

# Labels and title
plt.xlabel("Temperature)")
plt.ylabel("Average q6")
plt.title("Average q6 vs Temperature")
plt.grid(True)

# Show the plot
plt.show()

Compare your result with the top panel of Figure 2 of [this paper](https://aip.scitation.org/doi/pdf/10.1063/1.481671) and see if you recover the same trend

#### End Assignment 4

#### Assignment 5: heat capacity at constant volume

Statistical mechanics defines the **heat capacity at constant volume** $C_V$ in terms of the fluctuations of variance of energy compared to the square of the expectation value of energy:

$$
C_V = k_B\beta^2[\langle E^2 \rangle - \langle E \rangle^2].
$$

Plot this quantity vs. the temperature (your plot might look nicer, however, if you plot the normalised expectation value of $C_V$ vs the normalised temperature: $\langle C_V \rangle / Nk_B$ vs. $k_BT/\varepsilon$). Recover the plot in the top panel of FIG. 1 of the paper cited in Assignment 4.

**If you feel like it**: carry out the differentiation in Eq. (15) of the paper and reproduce the bottom panel of FIG. 1.

In [ ]:
## Your code here

In [ ]:
import matplotlib.pyplot as plt
from numpy.random import default_rng
import numpy as np
import torch

from utils import split_data, train, MLP
import torch.nn.functional as F
from torch import nn
from torch.utils.data import  DataLoader
from IPython.display import clear_output
import numpy as np
import mdtraj as md

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# check if a GPU is available
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

# Reading a torch object from an xyz

In [ ]:
import torch
import numpy as np

def load_extxyz(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    # Extract atomic positions (skip first two lines)
    data = [line.split()[1:4] for line in lines[2:]]

    # Convert to NumPy array and then to Torch tensor
    np_data = np.array(data, dtype=np.float32)
    tensor_data = torch.tensor(np_data)

    return tensor_data

clus3d = load_extxyz("T0.xyz")/sigma_lammps  #rescaling to lj units


ase_clus3d = read ("T0.xyz")

## MD step function

In [ ]:
def energy_lj(r: torch.tensor, epsilon: float = 1.0, sigma: float = 1.0):
    """Compute the Lennard-Jones energy of a system with positions r

    Parameters
    ----------
    r: atomic configuration

    sigma: LJ potential parameter sigma, defined elsewhere

    epsilon: LJ potential parameter epsilon, defined elsewhere

    Return
    ------
    ene: total LJ energy of atomic configuration

    """
    def lj(dist, epsilon: float = 1.0, sigma: float = 1.0):
        return 4 * epsilon * ((sigma / dist)**12 - (sigma / dist)**6)

    distances = F.pdist(r)

    pair_energies = torch.vmap(lj)(distances)
    return torch.sum(pair_energies)



def forces_lj(r: torch.Tensor, epsilon: float = 1.0, sigma: float = 1.0):
    """Compute the force of the Lennard-Jones potential of a system with positions r

    Parameters
    ----------
    r: atomic configuration


    sigma: LJ potential parameter sigma, defined elsewhere

    epsilon: LJ potential parameter epsilon, defined elsewhere

    Return
    ------
    f: atomic forces
    """

    n_particles = r.shape[0]

    # get vectors rij (N x N x 3)
    rij = r[:, None, :] - r[None, :, :]  # array with all rij = rj - ri
    # get minimum image vector

    # get distance matrix (N x N)
    dij = torch.linalg.norm(rij, axis=-1)
    # to avoid dividing by zero, set diagonal elements dii to a small value
    dij.fill_diagonal_(1e5)
    # compute matrix of fij forces with fij = -dU/d|rij| * 1/|rij|, (N x N)
    fij = (-24*epsilon/(dij**2) * ((sigma/dij)**6) * (2.0*(sigma/dij)**6 - 1.0))

    # set diagonal elements fii to zero
    fij.fill_diagonal_(0.0)
    # initialize force array (N x N x 3)
    fi = torch.zeros((n_particles, n_particles, 3))
    # compute total force
    for i in range(3):
        fi[:, :, i] = fij * rij[:, :, i]

    # compute sum over j: fi = \sum_j fij (N x 3)
    f = torch.sum(fi, axis=0)

    return f

In [ ]:
def get_forces(energy_fn, r: torch.Tensor):

    """Computes the forces acting on a configuration r for a given energy function using backpropagation.

    Parameters
    ----------
    r: atomic configuration

    energy_fn: energy function


    Return
    ------
    f: atomic forces
    """

    assert r.requires_grad == False

    r.requires_grad = True

    energy = energy_fn(r)
    energy.backward()

    forces = -r.grad
    r.requires_grad = False
    return forces

In [ ]:
r_test = clus3d


In [ ]:
energy = energy_lj(r_test)
forces = forces_lj(r_test)

In [ ]:
forces

In [ ]:
get_forces(energy_lj, r_test)

# read a trajectory

In [ ]:
n_files=4

temp = 20
filename1 = f"structure_{temp}.xyz"

temp = 30
filename2 = f"structure_{temp}.xyz"

temp = 5
filename3 = f"structure_{temp}.xyz"

temp = 15
filename4 = f"structure_{temp}.xyz"

traj = read(filename1,index="1::2")+read(filename2,index="1::2")+read(filename3,index="1::2")+read(filename4,index="1::2")

print (f"Trajectory for T={temp}")

for atoms in traj:
    atoms.positions=atoms.positions/sigma_lammps
    
print (len(traj))
sigma=1.0
view_trajectory (traj)



In [ ]:
sigma = 1.0
epsilon = 1.0

# now we go back to LJ units


# Initialize a list to store tensors
torch_tensors = []

# Loop through each frame in the trajectory
for frame in traj:
    positions = frame.get_positions()  # Extract atomic positions
    tensor = torch.tensor(positions, dtype=torch.float32)  # Convert to Torch tensor
    torch_tensors.append(tensor)  # Store tensor

# Example: Check the shape of the first tensor
print(f"First frame tensor shape: {torch_tensors[0].shape}")  # (num_atoms, 3) for XYZ coordinates


n_particles = len(torch_tensors[0])
n_dimensions = 3
print (f"N_particles= {n_particles}")

In [ ]:
# initialize number of MD steps
n_steps = 500000*n_files
n_log = 1000



# energies
potential_energies = torch.empty((n_steps // n_log ,1))
kinetic_energies = torch.empty((n_steps // n_log ,1))
positions = torch.empty((n_steps // n_log , n_particles, n_dimensions))

In [ ]:
for i,frame in enumerate(traj):
    r_ase = frame.get_positions()
    r = torch.tensor(r_ase, dtype=torch.float32)
    positions [i] = r
    potential_energies[i] = energy_lj(r)

    
print (energy_lj(r),sigma,epsilon)



In [ ]:
plt.plot(positions[:,3, ]);

In [ ]:
plt.plot(positions[:,6, ]);

In [ ]:
plt.hist(potential_energies.flatten());

In [ ]:
potential_energies.shape

print (potential_energies)

## Train net

In [ ]:
type(positions)

In [ ]:
features = torch.vmap(F.pdist)(positions)
features,_ = torch.sort(features,dim=-1)
features.shape

In [ ]:
dataset_all, dataset_train, dataset_test = split_data(features, potential_energies, train_fraction=0.75,device=DEVICE)

In [ ]:
dataset_train.x.shape

In [ ]:
trainloader = DataLoader(dataset_train,batch_size=33,shuffle=True)

In [ ]:
mlp = MLP([703,50,50,50,50,50,50,50,50,1])

In [ ]:
# criterion to computes the loss between input and target
criterion = nn.MSELoss()

# optimizer that will be used to update weights and biases (learning rate 0.001)
optim = torch.optim.Adam(mlp.parameters(),lr=1e-2)

In [ ]:
losses = []

In [ ]:
epochs = 500
log_interval = 100
for epoch in range(1, epochs + 1):
    loss = train(mlp, trainloader,optim, criterion,DEVICE)

    losses.append(loss)

    if epoch % log_interval == 0:
        print(f'Train Epoch: {epoch} Loss: {loss:.6f}')

        clear_output(wait=True)
        mydist = f"dist {epoch}"
        plt.plot(losses, label=mydist)
        plt.yscale('log')
        plt.legend()
        plt.show()
    # validate(model, lossv, testloader, criterion, accv)

In [ ]:
# criterion to computes the loss between input and target
criterion = nn.MSELoss()

# optimizer that will be used to update weights and biases (learning rate 0.0001)
optim = torch.optim.Adam(mlp.parameters(),lr=1e-3)

In [ ]:
epochs = 2000
log_interval = 100
for epoch in range(1, epochs + 1):
    loss = train(mlp, trainloader,optim, criterion,DEVICE)

    losses.append(loss)

    if epoch % log_interval == 0:
        print(f'Train Epoch: {epoch} Loss: {loss:.6f}')

        clear_output(wait=True)
        mydist = f"dist {epoch}"
        plt.plot(losses, label=mydist)
        plt.yscale('log')
        plt.legend()
        plt.show()
    # validate(model, lossv, testloader, criterion, accv)

In [ ]:
# get predictions for training and test set
with torch.no_grad():

    # on pure data
    y_pred =mlp(dataset_test.x)

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(y_pred,dataset_test.y, label='dist')
plt.scatter(dataset_test.y,dataset_test.y, label='dist')

plt.show()

In [ ]:
from torchmetrics import MeanSquaredError

In [ ]:
mse = MeanSquaredError()

In [ ]:
mse(y_pred,dataset_test.y )

In [ ]:
def energy_mlp(positions):
    features,_ = torch.sort(F.pdist(positions))
    energy = mlp(features)
    return energy

In [ ]:
positions.shape

In [ ]:
with torch.no_grad():
    pred = torch.vmap(lambda pos: energy_mlp(pos))(positions)

In [ ]:
plt.scatter(pred, potential_energies)
plt.scatter(potential_energies, potential_energies)

In [ ]:
# set Boltzmann constant
beta = 20.
m = 1.

dt = 0.00005




def assign_MBv(n_particles, beta, m: float = 1.0):
    """Assign Maxwell-Boltzmann distributed velocities.

    Parameters
    ----------
    v: velocity array

    beta: 1/ (kB * T)

    m: atomic mass


    Return
    ------
    v:     velocity array

    """

    v = torch.normal(mean=0, std=np.sqrt(1.0 / (beta * m)), size=(n_particles, 3))

    return v

In [ ]:
r = torch.clone(positions[267])
# r.requires_grad = True
# assign new velocities

print (energy_lj(r))
v = assign_MBv(n_particles, beta, m)
f = get_forces(energy_mlp, r)





In [ ]:
# initialize number of MD steps
n_steps = 20000



n_log = 25

# energies
energies_mlp = torch.empty((n_steps // n_log,1))
energies_mlp_lj = torch.empty((n_steps // n_log,1))
kinetic_energies_mlp = torch.empty((n_steps // n_log,1))
positions_mlp = torch.empty((n_steps // n_log, n_particles, n_dimensions))

In [ ]:
# MD step
rng = default_rng(23)

# compute kinetic energy
def e_kin(v, m=1.0):
    ekin =  torch.sum(0.5*m*v*v)
    return ekin


def verlet_step_fast(
    r: torch.tensor,
    v: torch.tensor,
    f: torch.tensor,
    energy_fn: callable,
    dt: float,
    m: float = 1.0,
):
    """velocity Verlet step

    Parameters
    ----------
    r: positions

    v: velocities

    f: forces

    dt: time step

    m: mass

    energy_fn: energy function

    Return
    ------
    r: updated position

    v: updated velocities

    f: updated forces


    """
    # Remove center of mass motion
    com = torch.mean(v, axis=0)
    v -= com

    # update velocity 1/2 step
    v += 0.5 * f / m * dt

    # update positions
    r = r + v * dt

    f = get_forces(energy_fn, r)

    # update velocity 1/2 step
    v += 0.5 * f / m * dt

    return r, v, f

In [ ]:
get_forces(energy_lj,r)

In [ ]:


# run velocity Verlet
for i in range(n_steps):
    if i % n_log == 0:
        print (i)
        positions_mlp[i // n_log] = r
        with torch.no_grad():
            energies_mlp[i// n_log] = energy_mlp(r)
            energies_mlp_lj[i// n_log] = energy_lj(r)
            kinetic_energies_mlp[i// n_log] = e_kin(v,m)

    r, v, f = verlet_step_fast(r, v, f,energy_mlp, dt, m)
#    r, v, f = verlet_step_fast(r, v, f,energy_lj, dt, m)


In [ ]:
plt.hist(energies_mlp.detach().numpy(), density=True, alpha=.5)
plt.hist(energies_mlp_lj.detach().numpy(), density=True, alpha=.5)
#plt.hist(potential_energies.detach().numpy(), density=True, alpha=.5);

In [ ]:
plt.plot(energies_mlp)

In [ ]:
plt.plot(energies_mlp_lj)

In [ ]:
from ase.io import Trajectory
atoms = ase_clus3d
atoms.set_positions(atoms.get_positions()/sigma_lammps)
trajectory = Trajectory("output.traj","w")

for position_tensor in positions_mlp:
    atoms.set_positions(position_tensor.numpy())
    trajectory.write(atoms)
"""
    # Compute **pairwise distances** using `torch.cdist`

    distances = torch.cdist(position_tensor, position_tensor)  # Shape (N, N)

    # Extract **upper triangular distances** (to avoid duplicate pairs & self-distances)
    i, j = torch.triu_indices(*distances.shape, offset=1)  # Get indices of upper triangle
    pairwise_distances = distances[i, j]

    # Plot histogram of distances
    plt.hist(pairwise_distances.numpy(), bins=10, edgecolor="black")

    # Labels
    plt.xlabel("Pairwise Distance")
    plt.ylabel("Frequency")
    plt.title("Histogram of Pairwise Distances")

    # Show plot
    plt.show()"""

distances = torch.cdist(position_tensor, position_tensor)  # Shape (N, N)

# Extract **upper triangular distances** (to avoid duplicate pairs & self-distances)
i, j = torch.triu_indices(*distances.shape, offset=1)  # Get indices of upper triangle
pairwise_distances = distances[i, j]

# Plot histogram of distances
plt.hist(pairwise_distances.numpy(), bins=10, edgecolor="black")

# Labels
plt.xlabel("Pairwise Distance")
plt.ylabel("Frequency")
plt.title("Histogram of Pairwise Distances")

# Show plot
plt.show()


In [ ]:

traj = read("output.traj",index=":")

In [ ]:
view_trajectory(traj)

In [ ]:
trajectory = traj
q4_values = QL(4,trajectory)
# Plot Q4 along the trajectory
plt.plot(q4_values, marker="o")
plt.xlabel("Frame Index")
plt.ylabel("Q4 Value")
title = f"Q4 Structural Parameter, 1/T = {beta}"
plt.title(title)
plt.show()

q6_values = QL(6,trajectory)
# Plot Q4 along the trajectory
plt.plot(q6_values, marker="o")
plt.xlabel("Frame Index")
plt.ylabel("Q6 Value")
title = f"Q6 Structural Parameter, 1/T = {beta}"
plt.title(title)
plt.show()